# Incident Response Playbook

In [ ]:
from ollama import Client
import os

In [ ]:
def generate_incident_response_playbook(threat_type, environment_details):
    """
    Generate an incident response playbook based on the provided threat type and environment details.
    """
    # Create the messages for the OpenAI API
    messages = [
        {"role": "system", "content": "You are an AI assistant helping to create an incident response playbook."},
        {"role": "user", "content": f"Create a detailed incident response playbook for handling a '{threat_type}' threat affecting the following environment: {environment_details}."}
    ]

    # Make the API call
    try:
        options = {
        "temperature": 0.7
        }
        client = Client(host=os.getenv("OLLAMA_CHAT"))
        response = client.chat(
        model="llama3",
        messages=messages,
        options=options,
        stream=False            
    )
        return response['message']['content'].strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# Get input from the user
threat_type = input("Enter the threat type: ")
environment_details = input("Enter environment details: ")

In [ ]:
# Generate the playbook
playbook = generate_incident_response_playbook(threat_type, environment_details)

# Print the generated playbook
if playbook:
    print("\nGenerated Incident Response Playbook:")
    print(playbook)
else:
    print("Failed to generate the playbook.")

# Log Analysis

In [ ]:
from langchain_community.llms import Ollama

import os
import re
import os
import numpy as np
import faiss

from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
ollama = Ollama(
    base_url='http://192.168.10.80:11434',
    model="llama3"
)

oembed = OllamaEmbeddings(base_url="http://192.168.10.80:11434", model="nomic-embed-text")

In [ ]:
def parse_raw_log_to_json(raw_log_path):
    #Parses a raw log file and converts it into a JSON format.
    # Regular expressions to match timestamps and event descriptions in the raw log
    timestamp_regex = r'\[\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\]'
    event_regex = r'Event: (.+)'

    json_data = []

    with open(raw_log_path, 'r') as file:
        for line in file:
            timestamp_match = re.search(timestamp_regex, line)
            event_match = re.search(event_regex, line)

            if timestamp_match and event_match:
                timestamp = timestamp_match.group().strip('[]')
                event_description = event_match.group(1)
                json_data.append({"Timestamp": timestamp, "Event": event_description})
    
    return json_data

In [ ]:
def get_embeddings(texts):
    embeddings = []
    for text in texts:
        embedding = oembed.embed_documents(text)
        
        embeddings.append(embedding[0])

    return np.array(embeddings)

In [ ]:
def create_faiss_index(embeddings):
    # Creates a FAISS index for a given set of embeddings.
    d = embeddings.shape[1]  # Dimensionality of the embeddings
    index = faiss.IndexFlatL2(d)
    index.add(embeddings.astype(np.float32))  # FAISS expects float32
    return index

In [ ]:
def analyze_logs_with_embeddings(log_data):
    # Define your templates and compute their embeddings
    suspicious_templates = ["Unauthorized access attempt detected", "Multiple failed login attempts"]
    normal_templates = ["User logged in successfully", "System health check completed"]
    suspicious_embeddings = get_embeddings(suspicious_templates)
    normal_embeddings = get_embeddings(normal_templates)

    # Combine all template embeddings and create a FAISS index
    template_embeddings = np.vstack((suspicious_embeddings, normal_embeddings))
    index = create_faiss_index(template_embeddings)

    # Labels for each template
    labels = ['Suspicious'] * len(suspicious_embeddings) + ['Normal'] * len(normal_embeddings)

    categorized_events = []

    for entry in log_data:
        # Fetch the embedding for the current log entry
        log_embedding = get_embeddings([entry["Event"]]).astype(np.float32)

        # Perform the nearest neighbor search with FAISS
        k = 1  # Number of nearest neighbors to find
        _, indices = index.search(log_embedding, k)

        # Determine the category based on the nearest template
        category = labels[indices[0][0]]
        categorized_events.append((entry["Timestamp"], entry["Event"], category))

    return categorized_events

In [ ]:
# Sample raw log file path
raw_log_file_path = 'sample_log_file.txt'

# Parse the raw log file into JSON format
log_data = parse_raw_log_to_json(raw_log_file_path)

# Analyze the logs
categorized_timeline = analyze_logs_with_embeddings(log_data)

# Print the categorized timeline
for timestamp, event, category in categorized_timeline:
    print(f"{timestamp} - {event} - {category}")